In [12]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "../src"))

In [13]:
import logging
import jax
import jax.numpy as np
import numpy as onp
import skbio.stats.composition as cmp
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import statsmodels.api as sm
from datetime import datetime

from simulate_data_fct import sim_IV_ilr_linear, sim_IV_ilr_nonlinear
from plot_fct import update_layout, img_path, update_layout_px, colours, plot_ilr_X_vs_Y
from run_methods_all import run_methods_confidence_interval
from plot_fct import plot_mse_results, plot_beta_results

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# set up logging file
logging.getLogger().setLevel(logging.INFO)

In [16]:
key = jax.random.PRNGKey(253)

# Linear Setup

## Data Generation

In [17]:
n = 1000
p = 3
V = cmp._gram_schmidt_basis(p)
instrument_strength = "strong"
savepath = os.path.join(img_path, "NonLinearSetUp")
num_inst = 2
c_X = np.array([2, 2])  # confounder multiplication to X
alpha0 = np.array([1, 1])  # INSTRUMENT intercept
alphaT = np.array([[4, 1],  # INSTRUMENT INTERCEPT FIrst Component of Microbiome
                    [-1, 3]])  # INSTRUMENT INTERCEPT Second Component of Microbiome
beta0 = 5
betaT = np.array([6, 2])
mu_c = -1
c_Y = 4

In [18]:
confounder, Z_sim, X_sim, Y_sim, X_star, Y_star = sim_IV_ilr_nonlinear(
            key,
            n=n,
            p=p,
            num_inst=num_inst,
            mu_c=mu_c,
            c_X=c_X,
            alpha0=alpha0,
            alphaT=alphaT,
            c_Y=c_Y,
            beta0=beta0,
            betaT=betaT,
            num_star=500)
X_sim_ilr = cmp.ilr(X_sim)
X_star_ilr = cmp.ilr(X_star)

## Data Visualization

In [19]:
fig = px.bar(pd.DataFrame(X_sim[:100, :], columns=["Microbe 1", "Microbe 2", "Microbe 3"]))
fig = update_layout_px(fig)
fig.update_xaxes(title="Sample number")
fig.update_yaxes(title="Percentage of microbiota in sample")
fig.write_image(os.path.join(savepath,
                             "Barplot_Sample_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [20]:
# Compute F-Statistics 
ZZ_sim = onp.array(sm.add_constant(Z_sim))
alphahat = np.linalg.inv(ZZ_sim.T@ZZ_sim)@ZZ_sim.T@X_sim_ilr
MSM = np.sum(((ZZ_sim@alphahat - np.mean(X_sim_ilr, axis=0))**2), axis=0)/(num_inst)
MSE = np.sum(((ZZ_sim@alphahat - X_sim_ilr)**2), axis=0)/(n-num_inst)
F = MSM / MSE
F

DeviceArray([148.2662 , 100.69248], dtype=float32)

In [21]:
df = pd.DataFrame(X_sim, columns=["Microbe 1", "Microbe 2", "Microbe 3"])
fig = px.scatter_ternary(df, a='Microbe 1', 
                         b='Microbe 2', 
                         c='Microbe 3',
                         color=Z_sim[:, 0].squeeze(),
                        labels={"color": "Instrument value"})

fig = update_layout_px(fig)
#ig.update_layout(gridcolor="gray")
fig.write_image(os.path.join(savepath,
                             "Ternaryplot_SampleZ1_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [22]:
fig = px.scatter_ternary(df, a='Microbe 1', 
                         b='Microbe 2', 
                         c='Microbe 3',
                         color=Z_sim[:, 1].squeeze(),
                        labels={"color": "Instrument value"})

fig = update_layout_px(fig)
fig.write_image(os.path.join(savepath,
                             "Ternaryplot_SampleZ2_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

In [47]:
fig = plot_ilr_X_vs_Y(X_sim_ilr, X_star_ilr, Y_sim, Y_star)
fig.update_layout(width=2000, height=500)
fig.write_image(os.path.join(savepath,
                             "Data_vs_trueeffect_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

# Diversity Approach

In [24]:
from plot_fct import plot_diversity_methods
from helper_fct import diversity
from run_methods_all import run_diversity_estimation_methods
div_shannon = diversity(X_sim, "shannon")
div_simpson = diversity(X_sim, "simpson")

In [25]:
# Shannon Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_shannon, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

fig = plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(savepath,
                             "ShannonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

INFO:absl:Setup matrices...


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     61.99
Date:                 Fr, 11 Jun 2021   Prob (F-statistic):           4.22e-26
Time:                        10:21:22   Log-Likelihood:                -1360.3
No. Observations:                1000   AIC:                             2727.
Df Residuals:                     997   BIC:                             2741.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5494      0.079      6.920      0.0

INFO:absl:Optimize lambda...
INFO:absl:Optimal lambda [3.29948981]...
INFO:absl:Optimize xi...
INFO:absl:Optimal xi [-6.14312614]...
INFO:absl:Predict treatment effect...


In [26]:
# Simpson Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_simpson, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(savepath,
                             "SimpsonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

INFO:absl:Setup matrices...
INFO:absl:Optimize lambda...


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     57.49
Date:                 Fr, 11 Jun 2021   Prob (F-statistic):           2.35e-24
Time:                        10:21:25   Log-Likelihood:                -1364.4
No. Observations:                1000   AIC:                             2735.
Df Residuals:                     997   BIC:                             2749.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5329      0.080      6.685      0.0

INFO:absl:Optimal lambda [-2.30991675]...
INFO:absl:Optimize xi...
INFO:absl:Optimal xi [-5.58297065]...
INFO:absl:Predict treatment effect...


# Confidence Intervals for higher dimensional methods

In [27]:
from run_methods_all import run_methods_all

In [28]:
today = datetime.now()
num_iteration = 50

path = os.getcwd()
mypath = os.path.join(path, "temp",
                      today.strftime("%Y_%m_%d_%H_%M") + "_NONLINEAR_ConfidenceInterval_"+str(instrument_strength)
                      +"_"+ str(num_iteration)+"_Microbiota_" +str(p))

if not os.path.isdir(mypath):
    os.makedirs(mypath)


# create logging file with relevant information
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filename=os.path.join(os.getcwd(), mypath, "configurations.log"),
                    filemode='w'
                    )

logging.info("n="+str(n)+", p="+str(p)+", instrument strength: "+instrument_strength)
logging.info("Number of runs: "+str(num_iteration))
logging.info("First Stage Specification: ")
logging.info("Number of Instruments: "+str(num_inst))
logging.info("Instrument Strength by F-Test: " +str(F))
logging.info("alpha0="+str(alpha0)+", alphaT="+str(alphaT)+", confounder multiplicator c_X="+str(c_X))
logging.info("Second Stage Specification: ")
logging.info("beta0="+str(beta0)+", betaT="+str(betaT)+", confounder multiplicator c_Y="+str(c_Y))
logging.info("Confounder mean mu_c="+str(mu_c))


df_beta, df_mse, mse_large_confidence= run_methods_confidence_interval(
    key, num_iteration, 
    n, p, num_inst, mu_c, c_X, alpha0, alphaT, c_Y, beta0, betaT, 
    is_nonlinear=True,
    num_star=200)

**************************************************************************************************
*****************************************We are at 0 of 50***********************************************
**************************************************************************************************


/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/dirichlet/dirichlet.py:265: RuntimeWarning:

Mean of empty slice.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in true_divide



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [3.03 2.33]
Estimated Beta: [ 3.09 -1.19 -1.9 ]
Error: 27.7

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Error in computation
True Beta: [6 2]
Estimated Beta: [ 3.09 -1.19 -1.9 ]
Error: 27.7

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Error in computation
Beta DirichLetLogContrast: 1.39[ 6.672 -4.526 -2.146]
True Beta: [6 2]
Estimated Beta: [7.92 2.63]
Error: 351.53

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/dirichlet/dirichlet.py:265: RuntimeWarning:

Mean of empty slice.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in true_divide



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [2.83 2.14]
Estimated Beta: [ 2.87 -1.13 -1.75]
Error: 20.95

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Error in computation
True Beta: [6 2]
Estimated Beta: [ 2.87 -1.13 -1.75]
Error: 20.95

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Error in computation
Beta DirichLetLogContrast: 1.962[ 5.371 -3.957 -1.415]
True Beta: [6 2]
Estimated Beta: [6.6  1.73]
Error: 183.32

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<

...

<Figure size 800x240 with 0 Axes>

<Figure size 800x240 with 0 Axes>

<Figure size 800x240 with 0 Axes>

<Figure size 800x240 with 0 Axes>

<Figure size 800x240 with 0 Axes>

In [29]:
import pickle
results = {
    "df_mse": df_mse,
    "df_beta": df_beta,
    "mse_large": mse_large_confidence
}

with open(os.path.join(savepath, "results_"+str(p)+str(instrument_strength)+".pickle"), "wb") as f:
    pickle.dump(results, f)
    f.close()


# Mean Squarred Error Plot

In [43]:
df_mse.Method = df_mse.Method.replace({"KIV+KIV":"KIV"})

In [45]:
filter_list = ["ALR+LC", "DIR+LC",  "ONLY Second LC", "KIV"]

fig = plot_mse_results(df_mse, filter_list, sort_to_filter=True)
#fig.update_yaxes(range=(0, 100))
fig.write_image(os.path.join(savepath,
                             "MSE_Small_NonLinear" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [40]:
filter_list = ["ALR+LC", "ILR+LC", "ILR+ILR", "DIR+LC",  "ONLY Second LC", "ONLY Second ILR"]

fig = plot_mse_results(df_mse, filter_list, sort_to_filter=True)
#fig.update_yaxes(range=(0, 100))
fig.write_image(os.path.join(savepath,
                             "MSE_Full_NonLinear" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
filter_list = ["ALR+LC", "ILR+LC", "ILR+ILR", "ONLY Second LC", "ONLY Second ILR"]

fig = plot_mse_results(df_mse, filter_list, sort_to_filter=True)
#fig.update_yaxes(range=(0, 100))
fig.write_image(os.path.join(savepath,
                             "MSE_Full_NonLinear" +str(p)+"InstrumentStrength_"+str(instrument_strength)+"_2.pdf" ))
fig.show()